<a href="https://colab.research.google.com/github/SpellOnYou/100_Days_of_ML_Code/blob/master/code/Neural_Network_Language_Model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

In [0]:
sents_kor = ["나는 이 강아지를 좋아한다", "나는 저 커피를 사랑한다", "나는 이 우유를 싫어한다"]
sents_eng = ["i this like dog", "i love that coffee", "i hate this milk", ]

In [0]:
sents_kor[0].split()[2:5]

['강아지를', '좋아한다']

In [0]:
def word_dict(sentences, idx = False):
  word_list = list(set(" ".join(sentences).split()))
  if not idx:
    return{word:idx for idx, word in enumerate(word_list)}
  return {idx:word for idx, word in enumerate(word_list)}  

In [0]:
bag_kor = word_dict(sents_kor); bag_eng = word_dict(sents_eng)
n_class_eng = len(bag_eng); n_class_kor = len(bag_kor)

In [0]:
print("korean |V|:{}, english |V|: {}".format(len(bag_kor), len(bag_eng)))

korean |V|:9, english |V|: 9


In [0]:
# NNLM Parametber
n_step = 2 # number of steps ['i like', 'like the', 'the dog'].., This is why people NNLM is n-gram mechanism.
n_hidden = 1 # number of hidden units

In [0]:
bag_kor

{'강아지를': 4,
 '나는': 5,
 '사랑한다': 2,
 '싫어한다': 1,
 '우유를': 7,
 '이': 8,
 '저': 6,
 '좋아한다': 3,
 '커피를': 0}

In [0]:
[w for w in sents_kor[0].split()[:-1]]

['나는', '이', '강아지를']

In [0]:
windows = [sents_kor[0].split()[i:(i+n_step)] for i in range(len(sents_kor[0].split())-n_step)]; windows

[['나는', '이'], ['이', '강아지를']]

In [0]:
temp_input = [[bag_kor[win]  for win in window] for window in windows]; print(temp_input)

[[3, 8], [8, 2]]


In [0]:
[np.eye(9)[i] for i in temp_input]

[array([[0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.]])]

#### To make this example simpler, I used the last word as the target word.

In [0]:
def make_batch(sentences, n_step, n_class):
  input_batch, target_batch = [], []
  for sent in sentences:
    words = sent.split()
    windows = [words[i:(i+n_step)] for i in range(len(words)-n_step)]
    batch = [[bag_kor[win]  for win in window] for window in windows]
    input = [bag_kor[w] for w in words[:-1]] # sentence, except the last word
    target = bag_kor[words[-1]] #last word, idx # the last word
    
    # one-hot encoding
    input_batch.append([np.eye(n_class)[i] for i in batch])
    target_batch.append(np.eye(n_class)[target])
  return input_batch, target_batch

In [0]:
BATCH_SIZE = len(sents_kor[0].split())-n_step

In [0]:
input_batch_kor, target_batch_kor = make_batch(sents_kor, n_step=2, n_class = n_class_kor)

In [0]:
X = tf.placeholder(tf.float32, [None, n_step * BATCH_SIZE, n_class_kor]) # [label, number of steps * batch, number of Vocabulary]
Y = tf.placeholder(tf.float32, [None, n_class_kor])

In [0]:
print(X)

Tensor("Placeholder:0", shape=(?, 2, 9), dtype=float32)


In [0]:
print(X) # after batchsize

Tensor("Placeholder:0", shape=(?, 4, 9), dtype=float32)


In [0]:
input = tf.reshape(X, shape=[-1, n_step * n_class_kor * BATCH_SIZE])

#### H, hidden layer, matrix size:  (n_step * n_class_kor, n_hidden)

In [0]:
H = tf.Variable(tf.random_normal([BATCH_SIZE * n_step * n_class_kor, n_hidden])); H

<tf.Variable 'Variable_4:0' shape=(36, 1) dtype=float32_ref>

#### b, Bias of hidden layer

In [0]:
d = tf.Variable(tf.random_normal([n_hidden])); d

<tf.Variable 'Variable_5:0' shape=(1,) dtype=float32_ref>

In [0]:
U = tf.Variable(tf.random_normal([n_hidden, n_class_kor])); U

<tf.Variable 'Variable_6:0' shape=(1, 9) dtype=float32_ref>

In [0]:
b = tf.Variable(tf.random_normal([n_class_kor])); b

<tf.Variable 'Variable_7:0' shape=(9,) dtype=float32_ref>

In [0]:
tanh = tf.nn.tanh(d + tf.matmul(input, H)) # [batch_size, n_hidden]
model = tf.matmul(tanh, U) + b # [batch_size, n_class]

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
prediction =tf.argmax(model, 1)

### Training

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
tf.reshape(X, shape=[-1, n_step*BATCH_SIZE , n_class_kor])

input_batch, target_batch = make_batch(sents_kor, n_step=2, n_class=n_class_kor)

tf.convert_to_tensor(input_batch)

help(np.reshape)

In [0]:
input_batch = np.reshape(input_batch, [-1, n_step * BATCH_SIZE, n_class_kor])

(?, 2, 9) 가 들어와야 하는데 (3,2,2,9)가 들어와서 에러. 이런 내가 문장 수를 늘리고 batch가 한 단계 깊어져서 그런듯?

그렇다면 위에서 텐서 리쉡을 해줘야 하는데 디멘젼을 어떻게 낮추는지 모르겠따 ㅠㅠㅠㅠㅠ

텐서 리셉이 아니라 넘파이 리셉을 해줘야 먹히는데 그 이유는 아직 모르겠다.

In [0]:
input_batch.shape

(3, 4, 9)

In [0]:
input_batch.shape

(6, 2, 9)

In [0]:
target_batch

[array([0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0.])]

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,2,9]
	 [[{{node Placeholder}}]]


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,2,9]
	 [[node Placeholder (defined at <ipython-input-12-940b22d6dea7>:1) ]]

In [0]:
for epoch in range(5000):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
    if (epoch + 1)%1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

Epoch: 1000 cost = 1.056529
Epoch: 2000 cost = 0.589887
Epoch: 3000 cost = 0.424599
Epoch: 4000 cost = 0.273124
Epoch: 5000 cost = 0.182294


### Predict

In [0]:
predict =  sess.run([prediction], feed_dict={X: input_batch})

In [0]:
predict

[array([3, 2, 7])]

### Test

In [0]:
input = [sen.split()[:2] for sen in sents_kor]

In [0]:
input

[['나는', '이'], ['나는', '저'], ['나는', '이']]

In [0]:
number_dict_kor = word_dict(sents_kor, idx=True)

In [0]:
print([sen.split()[:3] for sen in sents_kor], '->', [number_dict_kor[n] for n in predict[0]])

[['나는', '이', '강아지를'], ['나는', '저', '커피를'], ['나는', '이', '우유를']] -> ['좋아한다', '사랑한다', '싫어한다']
